In [378]:
# Linear Regression by Nomal Equation

import numpy as np
import matplotlib.pyplot as pl

def generateY(x, a= 1, b= 0):
    ϵ= np.random.normal(
        loc=   0.0, 
        scale= np.random.uniform(0,5), 
        size= x.shape)
    y= a*x +b +ϵ
    return y

np.random.seed(0)
N=    100
low= -3.0
high= 4.0
a=  4  # 斜率
b= -3  # y軸 截距

x= np.random.uniform(low= low, 
                     high=high, 
                     size=N)
y= generateY(x, a, b)

pl.scatter(x, y, color='red', label='Real data')
pl.axhline(color= 'gray')
pl.axvline(color= 'gray')
pl.xlabel('x')
pl.ylabel('y')
pl.title('$y=f(x)$')
pl.legend()
pl.grid()

# the Normal Equation

Φ= np.concatenate((np.ones(N).reshape(N, 1),
                   x.reshape(N, 1)), 
                   axis=1)
Y= y.reshape(N, 1)

w= np.linalg.inv(Φ.T @Φ) @ (Φ.T@Y)    # the Normal Equation is here           

w= np.squeeze(w) #只是把矩陣形狀重整一下，沒做也沒關係。

print("Real parameters used creating the data")
print(f"b=w0= {b:.4f}, a=w1= {a:.4f} ")
print("Exact Solution using the normal equation")
print(f"w0: {w[0]:.4f}  w1: {w[1]:.4f}")

Real parameters used creating the data
b=w0= -3.0000, a=w1= 4.0000 
Exact Solution using the normal equation
w0: -2.5703  w1: 3.8713


In [379]:
pl.figure()
pl.plot(x, y, 'ro', label='Real data')

y_gen= a * x + b
pl.plot(x, y_gen, label='generated line equation')

y_pred= w[1] * x + w[0]
pl.plot(x, y_pred, label='predicted by Normal Equation')

pl.axhline(color='gray')
pl.axvline(color='gray')
pl.legend()
pl.grid()
pl.show()

In [377]:
# HomeWorkCh04_02

def generateY(x, w):
    ϵ= np.random.normal(
        loc=   0.0, 
        scale= np.random.uniform(1,10), 
        size= x.shape[1])    
    x= np.concatenate(
            (np.ones((1,N)),
             x), 
            axis=0)
    y= w.T@x +ϵ
    return y

np.random.seed(0)
N=    100

b= -3  # y軸 截距

w= np.array([b, 3, -2]) #.reshape(-1,1)

x= np.random.uniform(low= low, 
                     high=high, 
                     size=(2,N))

y= generateY(x, w)


%matplotlib qt
ax= pl.axes(projection='3d')
ax.scatter3D(x[0,:],x[1,:],y, color='red')
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.set_zlabel('$y$')

Text(0.5, 0, '$y$')

In [155]:
# the Normal Equation

Φ= np.concatenate((np.ones((N,1)),
                   x.T), 
                   axis=1)
Φ.shape

Y= y.reshape(N,1)
Y.shape

w_pred= np.linalg.inv(Φ.T @Φ) @ (Φ.T@Y)    # the Normal Equation is here      
w_pred

w_pred= np.squeeze(w_pred) #把矩陣形狀重整一下
w_pred

print("Real parameters used creating the data")
print(f"w0= {w[0]:.4f}, w1= {w[1]:.4f}, w2= {w[2]:.4f} ")
print("Exact Solution using the normal equation")
print(f"w0: {w_pred[0]:.4f}  w1: {w_pred[1]:.4f}, w2: {w_pred[2]:.4f}")

pl.figure()

ax= pl.axes(projection='3d')
ax.scatter3D(x[0,:],x[1,:],y, color='red', label='Real data')
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.set_zlabel('$y$')

xx= np.concatenate(
        (np.ones((1,N)),
         x), 
        axis=0)
y_pred= w_pred.T@xx
    
ax.scatter3D(xx[1,:],xx[2,:],y_pred, color='green',  label='predicted data')
ax.legend()

x0mesh, x1mesh= np.mgrid[-3:4:.1, -3:4:.1]
ymesh= w_pred[0] + w_pred[1] *x0mesh +  w_pred[2] *x1mesh

ax.contour3D(x0mesh,x1mesh,ymesh, cmap='rainbow', levels=200) 


Real parameters used creating the data
w0= -3.0000, w1= 3.0000, w2= -2.0000 
Exact Solution using the normal equation
w0: -2.8707  w1: 2.9854, w2: -2.0260


In [380]:
#
# this class only do 1-d data
# 若要延伸至 2-d data, 還有很多地方要改....
#
class LinearRegression(object):
    
  def __init__(self, data_x, data_y,
               w_init=None, 
               b_init=None, 
               learning_rate=   1e-3,
               minLossDecrease= 1e-6):
    
    # a, b 須來自外部 global, 這樣不好！改...
    
    #scale = 4.0 #改後就用不到了
    
    
    if w_init is not None:
      self.w = w_init
    else:
      self.w = np.random.uniform(-1,1) #(low=a-scale, high=a+scale)
    
    if b_init is not None:
      self.b = b_init
    else:
      self.b = np.random.uniform(-1,1) #(low=b-scale, high=b+scale)
    print("w_init: {:.3f}".format(self.w))
    print("b_init: {:.3f}".format(self.b))
      
    self.x= data_x
    self.y= data_y
    self.lr = learning_rate
    
    # for accumulation of loss and path (w, b)
    
    # 這 3 行 移到 train() 裡面
    
    #self.loss_history = []
    #self.w_history = []
    #self.b_history = []
    
    self.minLossDecrease= minLossDecrease
  
  def inference(self, x, w=None, b= None):
    """Inference function for a linear model
      y_pred = w * x + b.
      
      # 這函數就是計算 prediction value，我也許會直接命名為 def predict()
    
    Args:
      x: full-batch data, shape: (1-rank Tensor (vector) np.array)
    
    Returns:
      y_pred: full-batch y_pred, shape: (1-rank Tensor (vector) np.array)
    
    """
    if w==None:
        w= self.w
    if b==None:
        b= self.b
    y_pred= w*x +b
    return y_pred
  
  def loss_for_plot(self, w, b):
    """List of loss function with respect to given list of (w, b).
      
    Args:
      w: shape: (1-rank Tensor (vector) np.array)
      b: shape: (1-rank Tensor (vector) np.array)
    
    Returns:
      loss_for_plot: shape: (1-rank Tensor (vector) np.array)
    """
    
    
    y_pred= np.matmul(
             np.expand_dims(self.x, axis=1), 
             np.expand_dims(w, axis=0)
            ) + b
    
    loss_for_plot= (y_pred - np.expand_dims(self.y, axis=1))**2
    loss_for_plot= np.sum(loss_for_plot, axis=0)
    
    '''
    y_pred=        self.inference(self.x, w, b)
    loss_for_plot= self.loss_fn(self.y, y_pred)
    '''
    return loss_for_plot
  
  def loss_fn(self, labels, predictions):
    """Loss function.
    MSE loss
    
    Args:
      labels: target data y, shape: (1-rank Tensor (vector) np.array)
      predictions: model inference y_pred, shape: (1-rank Tensor (vector) np.array)
    
    Returns:
      loss: mean value of loss for full-batch data, shape: (0-rank Tensor (scalar))
    """
    #loss = 0.5 * np.mean((predictions - labels)**2)
    
    # 嚴格來講，正確的理論形式應該如下：
    loss= np.sum((predictions - labels)**2)
    
    return loss
  
  def loss_derivative(self):
    """Loss derivative.
    
    Returns:
      dw: dL / dw, mean value of derivatives for full-batch data, shape: (0-rank Tensor (scalar))
      db: dL / db, mean value of derivatives for full-batch data, shape: (0-rank Tensor (scalar))
    """
    #dw= np.mean((self.y_pred - self.y) * self.x)
    #db= np.mean(self.y_pred - self.y)
    
    # 嚴格來講，正確的理論形式應該如下：
    dw= np.sum((self.y_pred - self.y) * self.x) *2
    db= np.sum(self.y_pred - self.y) *2
    
    # 但如果是這樣，它的值較大，要讓 learning rate 更小一點，否則每一步太大會發散！
    
    return dw, db
  
  def weights_update(self):
    """Weights update using Gradient descent.
    
      w' = w - lr * dL/dw
    """
    self.w +=  - self.lr * self.dw
    self.b +=  - self.lr * self.db
    
  def history_update(self, loss, w, b):
    """Accumulate all interesting variables
    """
    self.loss_history.append(loss)
    self.w_history.append(w)
    self.b_history.append(b)


  def train(self, max_epochs=100, minLossDecrease=None):
    
    if minLossDecrease==None:
        minLossDecrease= self.minLossDecrease
        
    # for accumulation of loss and path (w, b)
    self.loss_history = []
    self.w_history = []
    self.b_history = []
    
    pre_loss = 0.0
    for epoch in range(max_epochs):
      
      self.y_pred= self.inference(self.x) # 這行就是計算 prediction value
    
      self.loss=   self.loss_fn(self.y, self.y_pred)
      self.history_update(self.loss, self.w, self.b)
      
      if epoch % 10 == 0:
        print("epochs: {}  loss: {:.6f}  w: {:.5f}  b: {:.5f}"
              .format(epoch, self.loss, self.w, self.b))
      
      self.dw, self.db= self.loss_derivative()
      self.weights_update()
      
      # 此處安插 連續2次疊代loss值差異太小就終止疊代的額外處理
      if np.abs(pre_loss - self.loss) < minLossDecrease: #1e-6: 
        self.loss = self.loss_fn(self.y, self.y_pred)
        self.history_update(self.loss, self.w, self.b)
        print("epochs: {}  loss: {:.6f}  w: {:.5f}  b: {:.5f}"
              .format(epoch+1, self.loss, self.w, self.b))
        break
        
      pre_loss = self.loss
    
    # 跳出 for epoch 迴圈之外，就算訓練完成
    # 接下來只是一些 訓練過程的紀錄，方便我們觀察。
    self.w_history = np.array(self.w_history)
    self.b_history = np.array(self.b_history)
    self.path= np.concatenate(
        (np.expand_dims(self.w_history, 1), 
         np.expand_dims(self.b_history, 1)), 
        axis=1
        ).T

In [381]:
w_init= np.random.uniform(-5,5) #1.0
b_init= np.random.uniform(-5,5) #0.0
model= LinearRegression( x, y, 
                          w_init=w_init, 
                          b_init=b_init, 
                          #learning_rate= 0.001 #0.3 
                          # 微分項若改成 理論形式 (用sum 而非 mean)
                          # 則此處 learning rate 要求設定更小，
                          # 否則會發散)
                        )
model.w, model.b

w_init: -2.845
b_init: 4.474


(-2.8449232288644155, 4.473705904889243)

In [382]:
model.train(1000)
model.w, model.b, model.loss

epochs: 0  loss: 22079.136989  w: -2.84492  b: 4.47371
epochs: 10  loss: 1318.676025  w: 3.78600  b: -1.68839
epochs: 20  loss: 1243.547929  w: 3.86147  b: -2.46824
epochs: 30  loss: 1242.542353  w: 3.87021  b: -2.55846
epochs: 40  loss: 1242.528894  w: 3.87122  b: -2.56890
epochs: 50  loss: 1242.528714  w: 3.87133  b: -2.57010
epochs: 52  loss: 1242.528713  w: 3.87134  b: -2.57016


(3.8713378800254965, -2.5701593249573773, 1242.5287128372288)

In [383]:
y_pred= model.inference(x)
y_pred

array([ 6.88324583e-01,  5.19700483e+00,  2.15033187e+00,  5.81815381e-01,
       -2.70339686e+00,  3.31914746e+00, -2.32583734e+00,  9.98230922e+00,
        1.19304761e+01, -3.79315123e+00,  7.27107295e+00,  1.48543597e-01,
        1.20947402e+00,  1.08989083e+01, -1.22591409e+01, -1.18230243e+01,
       -1.36362672e+01,  8.37929627e+00,  6.90338098e+00,  9.39260393e+00,
        1.23357628e+01,  7.47251679e+00, -1.67837521e+00,  6.96767220e+00,
       -1.09790111e+01,  3.15728047e+00, -1.02993899e+01,  1.14157550e+01,
       -4.24147357e-02, -2.94709764e+00, -7.01488383e+00,  6.79706850e+00,
       -1.82278854e+00,  1.22002618e+00, -1.36749813e+01,  2.55335691e+00,
        2.40323254e+00,  2.53434670e+00,  1.13908008e+01,  4.29272429e+00,
       -4.44173709e+00, -2.34088446e+00,  4.72118956e+00, -1.25521009e+01,
        3.88478173e+00,  3.98968755e+00, -8.48293912e+00, -1.06903521e+01,
       -5.63626490e+00, -4.32784197e+00,  1.26779753e+00, -2.29835039e+00,
        1.26001306e+01, -

In [384]:
pl.scatter(x, y, label= 'generated data')
pl.scatter(x, y_pred, label='predicted data')
pl.grid()
pl.legend()

In [318]:
#Plot the loss function
pl.figure()
pl.title('Loss Function L')
pl.xlabel('Number of epochs')
pl.ylabel('Loss')
pl.plot(model.loss_history, label='gradient descent')
pl.legend()
pl.show()

In [319]:
#model.loss_history
model.path

array([[ 0.17282033,  3.1512193 ,  3.65331914,  3.75370529,  3.78583955,
         3.80392124,  3.81724176,  3.82777713,  3.83623634,  3.84304924,
         3.84853955,  3.85296457,  3.85653107,  3.85940564,  3.86172252,
         3.86358991,  3.865095  ,  3.8663081 ,  3.86728585,  3.8680739 ,
         3.86870907,  3.86922101,  3.86963362,  3.86996619,  3.87023424,
         3.87045028,  3.87062441,  3.87076475,  3.87087787,  3.87096905,
         3.87104253,  3.87110176,  3.87114949,  3.87118797,  3.87121898,
         3.87124397,  3.87126412,  3.87128036,  3.87129344,  3.87130399,
         3.87131249,  3.87131935,  3.87132487,  3.87132932,  3.87133291,
         3.8713358 ,  3.87133813],
       [-0.87208947, -0.98274304, -1.25566268, -1.50508408, -1.71083625,
        -1.87742745, -2.01181988, -2.12015852, -2.20748169, -2.27786394,
        -2.33459154, -2.38031353, -2.41716509, -2.44686714, -2.47080675,
        -2.49010187, -2.50565359, -2.51818814, -2.52829089, -2.53643362,
        -2.54299

In [385]:
# putting together our points to plot in a 3D plot
number_of_points = 100

margin = 2

w_min = model.w_history.min() - margin #-10 #a - margin
w_max = model.w_history.max() + margin #+10 #a + margin
b_min = model.b_history.min() - margin #-10 #b - margin
b_max = model.b_history.max() + margin #+10 #b + margin

w_points= np.linspace(w_min, w_max, number_of_points) 
b_points= np.linspace(b_min, b_max, number_of_points)
w_mesh, b_mesh = np.meshgrid(w_points, b_points)
loss_mesh = np.array(
    [model.loss_for_plot(wps, bps)
     #(wps) #, bps)
     for wps, bps in zip(w_mesh, b_mesh)])

In [328]:
w_mesh.shape, b_mesh.shape, loss_mesh.shape

((100, 100), (100, 100), (100, 100))

In [329]:
#model.loss_for_plot(w_mesh, b_mesh)
len(model.loss_history)


47

In [398]:
#model.loss_for_plot(w_mesh, b_mesh)
fig= pl.figure(figsize=(10, 8))
ax= pl.axes(projection='3d', elev=40, azim=-100)

#ax.plot_surface(w_mesh, b_mesh, loss_mesh, cmap='rainbow')
ax.contour(w_mesh, b_mesh, loss_mesh, levels=100, cmap='rainbow')


#'''
minima= (model.w, model.b)#W_exact.reshape(2, 1) # for 3D plot and contour plot



ax.plot(*minima, 
        model1.loss_for_plot(*minima), 
        'ro', 
        markersize=10)
#'''
path= model.path

ax.quiver(path[0,:-1], 
          path[1,:-1], 
          model.loss_for_plot(*path[:,:-1]),
          path[0,1:]-path[0,:-1], 
          path[1,1:]-path[1,:-1],
          model.loss_for_plot(*path[:,1:])- model.loss_for_plot(*path[:,:-1]),
          color= 'gray', 
          length= .2, 
          normalize= True,
          #scale_units= 'xy', 
          #angles= 'xy', 
          #scale= 1, 
          #color= 'blue'
         )

In [387]:
#from matplotlib.colors import LogNorm

#import matplotlib

fig, ax = pl.subplots(figsize=(10, 8))

ax.contour(w_mesh, 
           b_mesh, 
           loss_mesh, 
           levels= 100, #np.logspace(-1, 2, 35), 
           #norm= matplotlib.colors.LogNorm(), 
           cmap='rainbow')
ax.plot(*minima, 'ro', markersize=10)

ax.quiver(path[0,:-1], 
          path[1,:-1], 
          path[0,1:]-path[0,:-1], 
          path[1,1:]-path[1,:-1],
          scale_units= 'xy', 
          angles= 'xy', 
          scale= 1, 
          color= 'blue')

ax.set_xlabel('w')
ax.set_ylabel('b')

ax.set_xlim((w_min, w_max))
ax.set_ylim((b_min, b_max))

pl.grid()
pl.show()